In [2]:
import pandas as pd
import numpy as np
import requests


In [3]:
! pip install pandas-profiling

     -------------------------------------- 324.4/324.4 kB 3.3 MB/s eta 0:00:00
     -------------------------------------- 344.9/344.9 kB 7.1 MB/s eta 0:00:00
     -------------------------------------- 102.7/102.7 kB 6.2 MB/s eta 0:00:00
     -------------------------------------- 296.5/296.5 kB 9.2 MB/s eta 0:00:00
     -------------------------------------- 663.5/663.5 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 2.2/2.2 MB 9.3 MB/s eta 0:00:00
     ---------------------------------------- 4.7/4.7 MB 12.6 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27082 sha256=808cbd28bc7bc0af31de75f9fc43a86fc0ecd36ffe68093864e9876e94ede59c
  Stored in directory: c:\users\arsal\appdata\local\pip\cache\wheels\1d\05\04\c6d7d3b66539d9e659ac6dfe81e2d0fd4c1a8316cc5a403300
Successfully built htmlmin


In [4]:
df = pd.read_csv('titanic.csv')

In [6]:
from pandas_profiling import ProfileReport
prof = ProfileReport(df)
prof.to_file(output_file='profile.html')

C:\Users\arsal\AppData\Local\Temp\ipykernel_4680\2893370972.py:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
194,24,1,1,"Sloper, Mr. William Thompson",male,28.0,0,0,113788,35.5000,A6,S
593,822,1,3,"Lulic, Mr. Nikola",male,27.0,0,0,315098,8.6625,NaN,S
726,654,1,3,"O'Leary, Miss. Hanora ""Norah""",female,NaN,0,0,330919,7.8292,NaN,Q
630,462,0,3,"Morley, Mr. William",male,34.0,0,0,364506,8.0500,NaN,S
475,289,1,2,"Hosono, Mr. Masabumi",male,42.0,0,0,237798,13.0000,NaN,S


In [19]:
#Pclass is ordinal categorical variable,Sex is nominal CV as no relationship between categories in sex. 
df_new = df.loc[:,['Pclass','Sex','Embarked']]

In [21]:
df_new.sample(5)

,Pclass,Sex,Embarked
872,3,male,C
182,1,female,C
131,3,male,S
740,3,male,S
831,3,male,C


##### Ordinal encoding on ordinal categories with Label encoding on target label

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

In [25]:
x_train,x_test = train_test_split(df_new[['Pclass','Embarked']],test_size=0.1)

In [27]:
x_train['Embarked'].value_counts()

In [32]:
x_train.dropna(inplace=True)
x_train.isna().sum()

Pclass      0
Embarked    0
dtype: int64

In [28]:
oe = OrdinalEncoder(categories=[[1,2,3],['Q','C','S']]) #creating object with order of categories 1 is least,3 is highest

In [42]:
oe.categories_

[array([1, 2, 3], dtype=int64), array(['Q', 'C', 'S'], dtype=object)]

In [34]:
oe.fit(x_train)

OrdinalEncoder(categories=[[1, 2, 3], ['Q', 'C', 'S']])

In [35]:
x_train_encoded = oe.transform(x_train)

In [38]:
df_enc = pd.DataFrame(x_train_encoded,columns=x_train.columns)

In [39]:
df_enc.head()

,Pclass,Embarked
0,0.0,2.0
1,2.0,2.0
2,1.0,2.0
3,2.0,0.0
4,0.0,2.0


In [41]:
x_train.head()

,Pclass,Embarked
327,1,S
871,3,S
463,2,S
336,3,Q
20,1,S


##### One hot encoding on nominal categorical variable - creation of (n-1) dummy variables on n categories to avoid multicollinearity issue(dummy variable trap)

##### to encode categorical variable with high cardinality - we take most frequent values as distinct categories and bundle rest of the values under new category - Others

In [111]:
df = pd.read_csv('occupations.csv')

In [112]:
df['occupation'].value_counts()

student          196
other            105
educator          95
administrator     79
engineer          67
programmer        66
librarian         51
writer            45
executive         32
scientist         31
artist            28
technician        27
marketing         26
entertainment     18
healthcare        16
retired           14
lawyer            12
salesman          12
none               9
homemaker          7
doctor             7
Name: occupation, dtype: int64

In [113]:
df_new.nunique() ##21 different categories

21

In [79]:
df.sample(5)

,user_id,age,gender,occupation,zip_code
554,330,35,F,educator,33884
316,505,27,F,other,20657
431,637,30,M,other,74101
597,235,37,M,educator,22973
500,890,32,M,student,97301


In [114]:
df['age'].value_counts()
df['age'].nunique(),len(df) #61 unique ages out of 943 ppl.

(61, 943)

In [138]:
df['age'] = df['age'].astype('str')

In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     943 non-null    int64 
 1   age         943 non-null    object
 2   gender      943 non-null    object
 3   occupation  943 non-null    object
 4   zip_code    943 non-null    object
dtypes: int64(1), object(4)
memory usage: 37.0+ KB


### OneHotEncoding using Pandas

In [117]:
pd.get_dummies(df,columns=['age'],drop_first=True) #do drop_first to prevent prob of multicollinearity. for 61 categories,get 60 DVs (Total 4+60 columns)

,user_id,gender,occupation,zip_code,age_11,age_13,age_14,age_15,age_16,age_17,...,age_62,age_63,age_64,age_65,age_66,age_68,age_69,age_7,age_70,age_73
0,30,M,student,55436,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,471,M,student,77459,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,289,M,none,94619,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,142,M,other,48118,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,609,F,student,55106,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,585,M,librarian,98501,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
939,767,M,engineer,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
940,860,F,retired,48322,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
941,803,M,administrator,78212,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


##### we don't use pandas get_dummies in ML as pandas doesn't retain the order of columns which we need in ML

In [118]:
from sklearn.preprocessing import OneHotEncoder

In [127]:
ohe = OneHotEncoder(drop='first',sparse=False,dtype=np.int32)

In [129]:
occ_new = ohe.fit_transform(df[['occupation']])#.toarray()

In [131]:
occ_new

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [132]:
df_array = df[['user_id','age','gender','zip_code']].values

In [133]:
df_new = np.hstack((df_array,occ_new))
df_new

array([[30, '7', 'M', ..., 1, 0, 0],
       [471, '10', 'M', ..., 1, 0, 0],
       [289, '11', 'M', ..., 0, 0, 0],
       ...,
       [860, '70', 'F', ..., 0, 0, 0],
       [803, '70', 'M', ..., 0, 0, 0],
       [481, '73', 'M', ..., 0, 0, 0]], dtype=object)

In [135]:
df_new.shape ## columns 25 = 4 existing + 21 new dummy columns for occupation, after using drop first, 24 DVs

(943, 24)

### OneHotEncoding with top occupations

In [142]:
df['occupation'].value_counts() #21 different occupations,select top 8 rest mark uncommon

student          196
other            105
educator          95
administrator     79
engineer          67
programmer        66
librarian         51
writer            45
executive         32
scientist         31
artist            28
technician        27
marketing         26
entertainment     18
healthcare        16
retired           14
lawyer            12
salesman          12
none               9
homemaker          7
doctor             7
Name: occupation, dtype: int64

In [149]:
occupation_count = df['occupation'].value_counts()

In [150]:
occupation_count[occupation_count<45].index

Index(['executive', 'scientist', 'artist', 'technician', 'marketing',
       'entertainment', 'healthcare', 'retired', 'lawyer', 'salesman', 'none',
       'homemaker', 'doctor'],
      dtype='object')

In [151]:
pd.get_dummies(df['occupation'].replace(occupation_count[occupation_count<45].index,'Uncommon'))

,Uncommon,administrator,educator,engineer,librarian,other,programmer,student,writer
0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,1,0
2,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
938,0,0,0,0,1,0,0,0,0
939,0,0,0,1,0,0,0,0,0
940,1,0,0,0,0,0,0,0,0
941,0,1,0,0,0,0,0,0,0
